In [ ]:
import pandas as pd
import pandas_datareader as web
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import mplfinance as mpl
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import requests


def send_line_notify(notification_message):
    """
    LINEに通知する
    """
    line_notify_token = 'GlXSB9BURpdoAifXsABCBhAyNoehXTZ6Gp7Z2E8Wc2I'
    line_notify_api = 'https://notify-api.line.me/api/notify'
    headers = {'Authorization': f'Bearer {line_notify_token}'}
    data = {'message': f'message: {notification_message}'}
    requests.post(line_notify_api, headers = headers, data = data)
    


df = pd.read_csv('../symbols/japan_symbols.csv')
symbols = []
content = '\n\nToday`s stock report' + str(dt.date.today())
count = 0

print(df.head())

#----------------------------------------------------------------------------
for symbol in df['symbol'][750:800] :   
    x = str(symbol) + '.T'
#     print(x)
    symbols.append(x)

for symbol in symbols:
    start=dt.datetime.now()-dt.timedelta(days=90)
    end=dt.datetime.now()
    count += 1
    print(count)
    
    try:
        symbol_df = yf.download(symbol, start, end, interval='1d')
        market_cap = web.get_quote_yahoo(symbol)['marketCap']
        print('時価総額は'+ str(market_cap))
        code, score = str(market_cap).split()
#         type(market_cap)
    except:
        print('時価総額なし'+str(symbol))
        
    if symbol_df.empty or market_cap.empty:
        continue
        
    print(symbol)
    
    
    
    # high in the past 81 days
    df['Highest81'] = df['Adj Close'].rolling(window=81).max()
    # 標準偏差を計算
    try:
        short_sma = 25
        df['SMA'+str(short_sma)] = df['Adj Close'].rolling(window=short_sma).mean()
        df['STD'] = df['Adj Close'].rolling(window=25).std()
        df['Standard_deviation_normalization'] = 100 * 2 * df['STD'] / df['SMA'+str(short_sma)]


        highest = df['Highest81'][-2]
        close = df['Adj Close'][-1]
        std = df['Standard_deviation_normalization'][-1]
        
    except KeyError:
        continue

    if std < 3 and close > 0.96 * highest and market_cap > 30000000000:
        print('signal')
        content_x = "\n{} 👍 \n逆指値を入れる値段: ¥{}\n前の終値: ¥{}".format(symbol, round(highest, 5), round(close, 5))
        content += '\n'+content_x
        
#     else :
#         print('NO')
print(content)
# send_line_notify(content)